In [1]:
#Initialize dependencies
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorboard as tb
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.create_config import transformer_config, metrics_config
from modules.tf_text_postprocess import TextPostprocessor
from modules.datagen import DataGenerator

In [2]:
session_name='novelsdreamer_test_2'

# Config training

In [3]:
# Configure the Transformer layer
config = transformer_config()
config_metrics = metrics_config()

embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])
regularized_layer = RegularizedDenseLayer(config["d_model"])

model = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                          config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                          pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer, regularized_layer=regularized_layer)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='auto')
optimizer = tf.keras.optimizers.legacy.Adam(name='Adam',learning_rate=0.001,
                                     epsilon=1e-8, amsgrad=True,
                                     beta_1=0.9, beta_2=0.999)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name=config_metrics['accuracy_set']), 
           tf.keras.metrics.MeanSquaredError(name=config_metrics['mean_sq_error']), 
           tf.keras.metrics.Precision(thresholds=config_metrics['thresholds'], name=config_metrics['precision']),]

model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)



In [4]:
MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'train')
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'valid')

datagen = DataGenerator(MAIN_DATASET_DIR, TRAIN_DATASET_DIR, VALID_DATASET_DIR, session_name=session_name)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()

Train data info: 2 classes, 2 samples
Valid data info: 2 classes, 2 samples


# Start training

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
epochs = 10
model.fit_model(train_english=train_english, 
                         train_russian=train_russian,
                         valid_english=valid_english,
                         valid_russian=valid_russian,
                         epochs=epochs,
                         session_name=session_name,
                         batch_size=32, gradient_accumulation_steps=5,
                         shuffle=True,
                         save_model_each_epoch=True,
                         model_name='novelsdreamer-ru-t4m')

  0%|          | 0/10 [00:00<?, ?it/s]

INFO:tensorflow:Epoch 1 Loss 4.8064 Validation Loss 4.9785
INFO:tensorflow:Beam search result: [[[52], 1.24537201452603], [[106], 1.7736237314541807], [[103], 1.799550593946515]]
INFO:tensorflow:Epoch 1 finished.
INFO:tensorflow:Model of 1 epoch saved in novelsdreamer_test_2/results/epoch_models/novelsdreamer-ru-t4m_epoch_1


 10%|█         | 1/10 [00:15<02:22, 15.82s/it]

INFO:tensorflow:Epoch 2 Loss 4.9528 Validation Loss 4.9785
INFO:tensorflow:Beam search result: [[[52], 1.24537201452603], [[106], 1.7736237314541807], [[103], 1.799550593946515]]
INFO:tensorflow:Epoch 2 finished.


 10%|█         | 1/10 [00:22<03:22, 22.55s/it]


KeyboardInterrupt: 

In [ ]:
model.load_weights('path_to_saved_weights.h5')  # replace with the path to your saved weights
input_sentence = "your_input_sentence"  # replace with your input sentence
translated_sentence = model.translate(input_sentence)
print(translated_sentence)

